In [50]:
%load_ext autoreload
%autoreload 2
import os

from clfm import clfm, manual_step, utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
path_database = '/media/elea/ssd/CLFM_DB'
type_alignment = 'polarimetry'
comparer = clfm.CLFMProject(path_database, type_alignment)
comparer.prepare_imgs()

In [52]:
import cv2
for _, folder in comparer.parameters_dict.items():
    df = pd.read_csv(os.path.join(folder['folder_gt_tile'], 'manual_selection.txt'), sep="\t")
    df['xSource'] = df['xSource'] // 2
    df['ySource'] = df['ySource'] // 2

    shape_original = cv2.imread(os.path.join(folder['folder_data'], "original", f"pair_{folder['current_tile']}_fixed.png")).shape
    shape_after = cv2.imread(os.path.join(folder['folder_data'], f"pair_{folder['current_tile']}_fixed.png")).shape
    ratio_x = shape_original[1] / shape_after[1]
    ratio_y = shape_original[0] / shape_after[0]
    df['xTarget'] = df['xTarget'] // ratio_x
    df['yTarget'] = df['yTarget'] // ratio_y
    df.astype(int).to_csv(os.path.join(folder['folder_gt_tile'], 'manual_selection.txt'), index=False, sep = "\t")

    try:
        os.remove(os.path.join(folder['folder_gt_tile'], 'gt_map.tif'))
    except:
        pass

In [6]:
for key, value in comparer.parameters_dict.items():
    path_selection = os.path.join(value['folder_gt_tile'], 'manual_selection.txt')
    if not os.path.exists(path_selection):
        manual_selector = manual_step.PointSelector(
            comparer.images[key]['reference'],
            comparer.images[key]['moving'],
            path_selection = path_selection
        )
        manual_selector.run()

Left-click anywhere to define a pair (any order). Right-click to undo last. Press any key to finish.
